In [1]:
# from underthesea import word_tokenize
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import numpy as np
import random

2024-02-13 15:17:00.093141: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-13 15:17:00.093237: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-13 15:17:00.210029: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# đường dẫn tới file chứa dữ liệu câu hỏi và câu trả lời
data_path = '/kaggle/input/chat-simsimi-dataset/chatbotDataset.txt'

# đọc dữ liệu từ file
with open(data_path, 'r', encoding='utf8') as f:
    lines = f.read().split('\n')

In [3]:
len(lines)

24783

In [4]:
Tokenizer = tf.keras.preprocessing.text.Tokenizer
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences
Sequential = tf.keras.models.Sequential
Embedding = tf.keras.layers.Embedding
SimpleRNN = tf.keras.layers.SimpleRNN
Dense = tf.keras.layers.Dense
LSTM = tf.keras.layers.LSTM
Dropout = tf.keras.layers.Dropout

In [5]:
# tạo câu hỏi và câu trả lời dưới dạng chuỗi số
question = []
answer = []

for line in lines:
    if line:
        line = line.strip()
        parts = line.split(":")
        if len(parts) == 2:
          question.append(parts[0])
          answer.append('<start> ' + parts[1] + ' <end>')

In [6]:
# Tokenize the text
tokenizer = Tokenizer(char_level=False, lower=True)
tokenizer.fit_on_texts(question + answer)

vocab_size = len(tokenizer.word_index) + 1

# Convert text to sequences
encoder_input =  pad_sequences(tokenizer.texts_to_sequences(question), padding='post')
decoder_input = pad_sequences(tokenizer.texts_to_sequences(answer), padding='post')
decoder_output = tf.keras.utils.to_categorical(pad_sequences([decode_ans[1:] for decode_ans in tokenizer.texts_to_sequences(answer)], maxlen=decoder_input.shape[1], padding='post'), vocab_size)

In [7]:
print("encoder input shape:", encoder_input.shape)
print("decoder input shape:", decoder_input.shape)
print("encoder output shape:", decoder_output.shape)

encoder input shape: (24775, 68)
decoder input shape: (24775, 70)
encoder output shape: (24775, 70, 5021)


In [11]:
def create_model():
  hidden_units  = 258

  encoder_inputs = tf.keras.layers.Input(shape=( encoder_input.shape[1] , ))
  encoder_embedding = tf.keras.layers.Embedding( vocab_size, hidden_units , mask_zero=True ) (encoder_inputs)
  encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( hidden_units , return_state=True )( encoder_embedding )
  encoder_states = [ state_h , state_c ]

  decoder_inputs = tf.keras.layers.Input(shape=( decoder_input.shape[1] ,  ))
  decoder_embedding = tf.keras.layers.Embedding( vocab_size, hidden_units , mask_zero=True) (decoder_inputs)
  decoder_lstm = tf.keras.layers.LSTM( hidden_units , return_state=True , return_sequences=True , name='decoder_lstm')
  decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )

  decoder_dense = tf.keras.layers.Dense( vocab_size , activation=tf.keras.activations.softmax )
  output = decoder_dense ( decoder_outputs )

  model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
  model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy')

  return model

model = create_model()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 68)]                 0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 70)]                 0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, 68, 258)              1295418   ['input_3[0][0]']             
                                                                                                  
 embedding_3 (Embedding)     (None, 70, 258)              1295418   ['input_4[0][0]']             
                                                                                            

In [12]:
def data_generator(encoder_input, decoder_input, decoder_output, batch_size):
    num_samples = len(encoder_input)
    steps_per_epoch = num_samples // batch_size

    while True:
        for i in range(steps_per_epoch):
            start = i * batch_size
            end = (i + 1) * batch_size

            encoder_input_batch = encoder_input[start:end]
            decoder_input_batch = decoder_input[start:end]
            decoder_output_batch = decoder_output[start:end]

            yield ([encoder_input_batch, decoder_input_batch], decoder_output_batch)

In [ ]:
# Train the model
epochs = 1000  # Increase the number of epochs to give the model more time to learn
batch_size = 2048

from tensorflow.keras.callbacks import ModelCheckpoint

# Định nghĩa callback để lưu mô hình có val_loss tốt nhất
checkpoint_filepath = 'best_model.h5df'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    monitor='loss',
    mode='min',
    verbose=1
)

# train_val_split = int(len(encoder_input)*0.1)
# train_data_generator = data_generator(encoder_input[train_val_split:], decoder_input[train_val_split:], decoder_output[train_val_split:], batch_size)
# val_data_generator = data_generator(encoder_input[:train_val_split], decoder_input[:train_val_split], decoder_output[:train_val_split], batch_size)

# history = model.fit_generator(generator=train_data_generator,
#                               steps_per_epoch=len(encoder_input[train_val_split:]) // batch_size,
#                               validation_data=val_data_generator,
#                               validation_steps=len(encoder_input[:train_val_split]) // batch_size,
#                               epochs=epochs,
#                               callbacks=[model_checkpoint_callback]
#                              )

train_data_generator = data_generator(encoder_input, decoder_input, decoder_output, batch_size)
history = model.fit_generator(generator=train_data_generator,
                              steps_per_epoch=len(encoder_input) // batch_size,
                              epochs=epochs,
                              callbacks=[model_checkpoint_callback]
                             )

/tmp/ipykernel_34/937938341.py:30: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator=train_data_generator,


Epoch 1/1000


I0000 00:00:1707837544.773915      97 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


12/12 [==============================] - ETA: 0s - loss: 8.4053
Epoch 1: loss improved from inf to 8.40534, saving model to best_model.h5
12/12 [==============================] - 58s 4s/step - loss: 8.4053
Epoch 2/1000


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


12/12 [==============================] - ETA: 0s - loss: 6.2810
Epoch 2: loss improved from 8.40534 to 6.28101, saving model to best_model.h5
12/12 [==============================] - 39s 3s/step - loss: 6.2810
Epoch 3/1000
12/12 [==============================] - ETA: 0s - loss: 5.3488
Epoch 3: loss improved from 6.28101 to 5.34876, saving model to best_model.h5
12/12 [==============================] - 39s 3s/step - loss: 5.3488
Epoch 4/1000
12/12 [==============================] - ETA: 0s - loss: 5.2151
Epoch 4: loss improved from 5.34876 to 5.21506, saving model to best_model.h5
12/12 [==============================] - 39s 3s/step - loss: 5.2151
Epoch 5/1000
 6/12 [==============>...............] - ETA: 19s - loss: 5.1487

In [10]:
model = tf.keras.models.load_model('/kaggle/working/best_model.h5')

# In thông tin về mô hình
model.summary()

OSError: No file or directory found at /kaggle/working/best_model.h5

In [ ]:
hidden_units = 1028

encoder_inputs = model.input[0]
encoder_states = model.layers[4].output[1:]
decoder_lstm = model.layers[5]
decoder_embedding = model.layers[3].output
decoder_dense = model.layers[6]
decoder_inputs = model.input[1]

In [12]:
def make_inference_models():

    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)

    decoder_state_input_h = tf.keras.layers.Input(shape=( hidden_units ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( hidden_units ,))

    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)

    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)

    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)

    return encoder_model , decoder_model

def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()

    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] )
    return pad_sequences( [tokens_list] , maxlen=encoder_input.shape[1] , padding='post')

In [15]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word

        if sampled_word == 'end' or len(decoded_translation.split()) >  decoder_input.shape[1]:
            stop_condition = True

        empty_target_seq = np.zeros( ( 1 , 1 ) )
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ]

    print( decoded_translation )

Enter question :  start


1/1 [==============================] - 0s 20ms/step
 required parameter is not present end


Enter question :  ngon


1/1 [==============================] - 0s 19ms/step
 ngon vcl end


Enter question :  chưa thấy ai ngon như mày


...

1/1 [==============================] - 0s 19ms/step
 kêu má a gãy hộ end


Enter question :  ơ đcm


1/1 [==============================] - 0s 19ms/step
 mày end


Enter question :  iw


KeyError: 'iw'